<a href="https://colab.research.google.com/github/kuberiitb/artificial_intelligence/blob/main/EvaluatingRAGSystemWithRagas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai>=1.0.0 ragas==0.2.10 langchain langchain_community langchain-openai langgraph --quiet

In [5]:
import os

import pickle
from datasets import Dataset
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

import ragas
from ragas.metrics import faithfulness, answer_correctness
from ragas import evaluate

In [3]:
ragas.__version__

'0.2.10'

In [4]:

load_dotenv(".env")

True

In [7]:
news_docs = [
"""Guinea-Bissau Coup: Soldiers in the West African nation announced they have taken “total control,” arresting President Embalo and suspending the electoral process. Military sources claim the detained president is being “well-treated,” while soldiers patrol capital streets. International reactions include calls for constitutional order and urgent humanitarian oversight.""",

"""Nigeria Freed 24 Kidnapped Students: Authorities announced the safe release of 24 students abducted from a school in Nigeria. The students, aged 12–17, were found unharmed in a rural region three days after being taken. Local police credit rapid intelligence and community cooperation for the rescue.""",

"""International Monetary Fund (IMF) has reclassified India’s exchange-rate regime from “stabilised” to a “crawl-like arrangement,” signifying increased flexibility and volatility in the rupee. The shift follows a six-month trend where the rupee remained near a statistical baseline but failed to meet floating-rate criteria. The IMF also projected India’s GDP growth at 6.6% for FY2025-26 and 6.2% the next year.""",

"""India to Host 2030 Centenary Commonwealth Games: At a general assembly in Glasgow, member nations ratified India’s city of Amdavad as the host for the 2030 Commonwealth Games. The edition will mark 100 years since the first Games. The sports program is expected to include athletics, swimming, boxing, gymnastics and possibly archery and badminton.""",

"""Renewed Hope for Indian Athletics: Athletics Federation of India (AFI) announced that India will host its first-ever World Athletics Continental Tour Silver-level meet next August in Bhubaneswar. The upgrade from a Bronze-level event is expected to draw international athletes and provide Indian competitors with valuable ranking points on home soil.""",

"""Markets Rally Ahead of Thanksgiving: The major U.S. stock indexes — the Nasdaq Composite, S&P 500 and Dow Jones Industrial Average — posted gains of 0.8%, 0.7%, and 0.7% respectively, extending a four-day rally as markets brace for the Thanksgiving holiday break. The Dow added 315 points in the session.""",

"""Global Climate Talks Wrap Up at 2025 United Nations Climate Change Conference (COP30) in Belém: After intense negotiations, oil-producing nations blocked binding commitments, leading the conference to adopt two voluntary “roadmaps” instead. The outcome disappointed many climate activists, especially indigenous groups who had protested for stronger action. A fire at the conference pavilion triggered a temporary evacuation — though no injuries were reported.""",

"""Disappointing Finish for India at ISSF World Championship Rifle/Pistol 2025: The event in Cairo saw India clinching 13 medals (3 gold, 6 silver, 4 bronze). Gold came from Men’s 10m Air Pistol (individual and team) and Men’s 50m Standard Pistol. Shooter Anish Bhanwala won a silver in Men’s 25m Rapid Fire, becoming the first Indian to medal in that Olympic-event pistol category at the Worlds.""",

"""Chess World Cup 2025 concludes in Goa: The 206-player knockout tournament ended with 19-year-old Javokhir Sindarov crowned champion. Runners-up included Wei Yi and Andrey Esipenko. All three secured spots in the 2026 Candidates Tournament, boosting their chances at the world championship cycle.""",

"""Middle-East Interfaith Diplomacy: Pope Leo XIV began a six-day trip to Turkey and Lebanon, marking his first overseas journey. The trip aims to foster Christian-Muslim unity and address migration, refugee welfare and declining Christian populations in the region. He plans to meet national leaders and religious dignitaries, and to deliver messages of peace and cooperation.""",

"""Ukraine Peace Moves Under Scrutiny: European leaders responded critically to a renewed peace proposal — reportedly shaped by a U.S. envoy — suggesting border changes in Ukraine. Vladimir Putin, whose forces are engaged in the conflict, acknowledged reviewing parts of the plan. Still, several officials demanded any deal must ensure a “just and lasting” peace rather than mere concessions.""",

"""G20 Summit Ends with Focus on Global South: The 2025 2025 G20 Johannesburg Summit, the first held on African soil, concluded with a 122-point declaration emphasizing global equity, debt relief for low-income nations, food security and multilateral reform. The decision signals a shift toward inclusive development and recognition of structural challenges faced by emerging economies.""",

"""HK High-Rise Blaze: A massive fire engulfed a high-rise residential complex in Hong Kong, killing at least 44 people and leaving hundreds missing. Rescue operations are ongoing. Preliminary investigations point to flammable bamboo scaffolding and mesh — traditional building aids — as a possible factor in the blaze’s rapid spread.""",

"""US Domestic Security Alert: Authorities in Washington, D.C. stopped processing immigration requests from Afghan nationals after two National Guard members were shot near the White House. The suspected gunman, an Afghan national, was taken into custody. The incident triggered increased security and a temporary pause on asylum processing.""",

"""Gold Prices Surge as Investors Eye Fed Rate Cuts: Spot gold climbed about 1% to reach its highest level in two weeks, driven by weak U.S. economic data and growing expectations that the Federal Reserve (Fed) may soon lower interest rates, prompting investors to seek the metal as a safe-haven asset.""",

"""Taiwan’s Defence Budget Hike: The government in Taipei announced plans to increase defence spending by US$ 40 billion to counter escalated pressure from neighbouring China. The announcement comes amid rising tensions in the Taiwan Strait and growing concern over regional security dynamics.""",

"""Sports Spotlight: 5th Khelo India University Games 2025 underway in Rajasthan — with athletes from across the country competing in 24 sports. The games aim to provide a national platform for university-level talent and spotlight sports such as athletics, badminton, hockey, wrestling and kabaddi.""",

"""Lottery Luck in UK: The latest draw of the National Lottery (UK) announced the winning numbers: 10, 21, 49, 54, 55, 56 (with bonus ball 40). The Thunderball numbers drawn were 15, 27, 30, 37, 39, with Thunderball 10. The jackpot prize stood at £2 million, reviving dreams of instant wealth for hopeful players.""",

"""Cinema Celebration in Goa: The ongoing 56th International Film Festival of India features over 240 films from 81 countries and celebrates Japan as its “Country of Focus.” The opening ceremony was organized as a carnival parade — a first for the festival. Several world-premiere films and masterclasses by prominent directors mark this edition as especially ambitious.""",

"""Tech and Telecom Crackdown Success: The national portal Sanchar Saathi, launched by the government’s telecom department, helped recover over 50,000 stolen mobile phones across India in October alone. Monthly recoveries surged 47% between June and October 2025, with states like Karnataka and Telangana each reporting over 100,000 recoveries since the portal’s inception.""",
]


In [8]:
qa_pairs = [
    {
        'question': "Who did the soldiers arrest in the West African nation in the recent coup?",
        'ground_truth': "They arrested President Embalo. Soldiers announced they had taken “total control” and arrested him. ",
        'context': """Guinea-Bissau Coup: Soldiers in the West African nation announced they have taken “total control,” arresting President Embalo and suspending the electoral process. Military sources claim the detained president is being “well-treated,” while soldiers patrol capital streets. International reactions include calls for constitutional order and urgent humanitarian oversight."""
    },
    {
        'question': "What did the military in Guinea-Bissau do after declaring total control?",
        'ground_truth': "They suspended the electoral process. ",
        'context': """Guinea-Bissau Coup: Soldiers in the West African nation announced they have taken “total control,” arresting President Embalo and suspending the electoral process. Military sources claim the detained president is being “well-treated,” while soldiers patrol capital streets. International reactions include calls for constitutional order and urgent humanitarian oversight."""
    },
    {
        'question': "How many students were freed in the Nigerian school abduction case?",
        'ground_truth': "24 students were freed. ",
        'context': """Nigeria Freed 24 Kidnapped Students: Authorities announced the safe release of 24 students abducted from a school in Nigeria. The students, aged 12–17, were found unharmed in a rural region three days after being taken. Local police credit rapid intelligence and community cooperation for the rescue."""
    },
    {
        'question': "After how many days were the kidnapped Nigerian students rescued?",
        'ground_truth': "They were rescued three days after being taken. ",
        'context': """Nigeria Freed 24 Kidnapped Students: Authorities announced the safe release of 24 students abducted from a school in Nigeria. The students, aged 12–17, were found unharmed in a rural region three days after being taken. Local police credit rapid intelligence and community cooperation for the rescue."""
    },
    {
        'question': "What change did the IMF make regarding India’s exchange-rate regime?",
        'ground_truth': "The IMF reclassified India’s exchange-rate regime from “stabilised” to a “crawl-like arrangement,” implying increased flexibility and volatility in the rupee. ",
        'context': """International Monetary Fund (IMF) has reclassified India’s exchange-rate regime from “stabilised” to a “crawl-like arrangement,” signifying increased flexibility and volatility in the rupee. The shift follows a six-month trend where the rupee remained near a statistical baseline but failed to meet floating-rate criteria. The IMF also projected India’s GDP growth at 6.6% for FY2025-26 and 6.2% the next year."""
    },
    {
        'question': "What GDP growth projections did the IMF give for India for FY2025-26 and the next year?",
        'ground_truth': "6.6% for FY2025-26 and 6.2% for the following year. ",
        'context': """International Monetary Fund (IMF) has reclassified India’s exchange-rate regime from “stabilised” to a “crawl-like arrangement,” signifying increased flexibility and volatility in the rupee. The shift follows a six-month trend where the rupee remained near a statistical baseline but failed to meet floating-rate criteria. The IMF also projected India’s GDP growth at 6.6% for FY2025-26 and 6.2% the next year."""
    },
    {
        'question': "Which city was ratified to host the 2030 Centenary Commonwealth Games?",
        'ground_truth': "The city of Amdavad in India was ratified to host the 2030 Commonwealth Games. ",
        'context': """India to Host 2030 Centenary Commonwealth Games: At a general assembly in Glasgow, member nations ratified India’s city of Amdavad as the host for the 2030 Commonwealth Games. The edition will mark 100 years since the first Games. The sports program is expected to include athletics, swimming, boxing, gymnastics and possibly archery and badminton."""
    },
    {
        'question': "What sports are expected to be included in the 2030 Commonwealth Games hosted by India?",
        'ground_truth': "The sports program is expected to include athletics, swimming, boxing, gymnastics and possibly archery and badminton. ",
        'context': """India to Host 2030 Centenary Commonwealth Games: At a general assembly in Glasgow, member nations ratified India’s city of Amdavad as the host for the 2030 Commonwealth Games. The edition will mark 100 years since the first Games. The sports program is expected to include athletics, swimming, boxing, gymnastics and possibly archery and badminton."""
    },
    {
        'question': "What event did the Athletics Federation of India announce for Bhubaneswar next August?",
        'ground_truth': "They announced that India will host its first-ever World Athletics Continental Tour Silver-level meet in Bhubaneswar next August. ",
        'context': """Renewed Hope for Indian Athletics: Athletics Federation of India (AFI) announced that India will host its first-ever World Athletics Continental Tour Silver-level meet next August in Bhubaneswar. The upgrade from a Bronze-level event is expected to draw international athletes and provide Indian competitors with valuable ranking points on home soil."""
    },
    {
        'question': "What is the significance of Silver-level status for the Bhubaneswar athletics meet?",
        'ground_truth': "The upgrade from a Bronze-level event is expected to draw international athletes and provide Indian competitors with valuable ranking points on home soil. ",
        'context': """Renewed Hope for Indian Athletics: Athletics Federation of India (AFI) announced that India will host its first-ever World Athletics Continental Tour Silver-level meet next August in Bhubaneswar. The upgrade from a Bronze-level event is expected to draw international athletes and provide Indian competitors with valuable ranking points on home soil."""
    },
    {
        'question': "Which US stock indexes gained ahead of Thanksgiving according to the markets report?",
        'ground_truth': "The Nasdaq Composite, S&P 500 and Dow Jones Industrial Average all posted gains ahead of Thanksgiving. ",
        'context': """Markets Rally Ahead of Thanksgiving: The major U.S. stock indexes — the Nasdaq Composite, S&P 500 and Dow Jones Industrial Average — posted gains of 0.8%, 0.7%, and 0.7% respectively, extending a four-day rally as markets brace for the Thanksgiving holiday break. The Dow added 315 points in the session."""
    },
    {
        'question': "By how much did the Dow increase in the session ahead of Thanksgiving?",
        'ground_truth': "The Dow added 315 points. ",
        'context': """Markets Rally Ahead of Thanksgiving: The major U.S. stock indexes — the Nasdaq Composite, S&P 500 and Dow Jones Industrial Average — posted gains of 0.8%, 0.7%, and 0.7% respectively, extending a four-day rally as markets brace for the Thanksgiving holiday break. The Dow added 315 points in the session."""
    },
    {
        'question': "What was the outcome of the 2025 climate conference (COP30) in Belém regarding binding commitments?",
        'ground_truth': "Oil-producing nations blocked binding commitments, so the conference adopted two voluntary “roadmaps” instead. ",
        'context': """Global Climate Talks Wrap Up at 2025 United Nations Climate Change Conference (COP30) in Belém: After intense negotiations, oil-producing nations blocked binding commitments, leading the conference to adopt two voluntary “roadmaps” instead. The outcome disappointed many climate activists, especially indigenous groups who had protested for stronger action. A fire at the conference pavilion triggered a temporary evacuation — though no injuries were reported."""
    },
    {
        'question': "Was anyone injured when a fire broke out at the COP30 pavilion?",
        'ground_truth': "No, there were no injuries reported when the pavilion was evacuated. ",
        'context': """Global Climate Talks Wrap Up at 2025 United Nations Climate Change Conference (COP30) in Belém: After intense negotiations, oil-producing nations blocked binding commitments, leading the conference to adopt two voluntary “roadmaps” instead. The outcome disappointed many climate activists, especially indigenous groups who had protested for stronger action. A fire at the conference pavilion triggered a temporary evacuation — though no injuries were reported."""
    },
    {
        'question': "How many total medals did India win at the 2025 ISSF World Championship Rifle/Pistol event in Cairo?",
        'ground_truth': "India won a total of 13 medals (3 gold, 6 silver, 4 bronze). ",
        'context': """Disappointing Finish for India at ISSF World Championship Rifle/Pistol 2025: The event in Cairo saw India clinching 13 medals (3 gold, 6 silver, 4 bronze). Gold came from Men’s 10m Air Pistol (individual and team) and Men’s 50m Standard Pistol. Shooter Anish Bhanwala won a silver in Men’s 25m Rapid Fire, becoming the first Indian to medal in that Olympic-event pistol category at the Worlds."""
    },
    {
        'question': "Which event did India win gold medals in at the 2025 ISSF Worlds in Cairo?",
        'ground_truth': "They won gold in Men’s 10m Air Pistol (individual and team) and Men’s 50m Standard Pistol. ",
        'context': """Disappointing Finish for India at ISSF World Championship Rifle/Pistol 2025: The event in Cairo saw India clinching 13 medals (3 gold, 6 silver, 4 bronze). Gold came from Men’s 10m Air Pistol (individual and team) and Men’s 50m Standard Pistol. Shooter Anish Bhanwala won a silver in Men’s 25m Rapid Fire, becoming the first Indian to medal in that Olympic-event pistol category at the Worlds."""
    },
    {
        'question': "Who won the 2025 Chess World Cup held in Goa?",
        'ground_truth': "19-year-old Javokhir Sindarov was crowned champion. ",
        'context': """Chess World Cup 2025 concludes in Goa: The 206-player knockout tournament ended with 19-year-old Javokhir Sindarov crowned champion. Runners-up included Wei Yi and Andrey Esipenko. All three secured spots in the 2026 Candidates Tournament, boosting their chances at the world championship cycle."""
    },
    {
        'question': "Which players secured spots in the 2026 Candidates Tournament from the 2025 Chess World Cup?",
        'ground_truth': "Javokhir Sindarov, Wei Yi and Andrey Esipenko secured spots in the 2026 Candidates Tournament. ",
        'context': """Chess World Cup 2025 concludes in Goa: The 206-player knockout tournament ended with 19-year-old Javokhir Sindarov crowned champion. Runners-up included Wei Yi and Andrey Esipenko. All three secured spots in the 2026 Candidates Tournament, boosting their chances at the world championship cycle."""
    },
    {
        'question': "What is the purpose of the Pope’s trip to Turkey and Lebanon described in the news?",
        'ground_truth': "The trip aims to foster Christian-Muslim unity and address migration, refugee welfare and declining Christian populations in the region. ",
        'context': """Middle-East Interfaith Diplomacy: Pope Leo XIV began a six-day trip to Turkey and Lebanon, marking his first overseas journey. The trip aims to foster Christian-Muslim unity and address migration, refugee welfare and declining Christian populations in the region. He plans to meet national leaders and religious dignitaries, and to deliver messages of peace and cooperation."""
    },
    {
        'question': "How long is the Pope’s overseas journey to Turkey and Lebanon?",
        'ground_truth': "The journey is six days long. ",
        'context': """Middle-East Interfaith Diplomacy: Pope Leo XIV began a six-day trip to Turkey and Lebanon, marking his first overseas journey. The trip aims to foster Christian-Muslim unity and address migration, refugee welfare and declining Christian populations in the region. He plans to meet national leaders and religious dignitaries, and to deliver messages of peace and cooperation."""
    },
    {
        'question': "What triggered the criticism from European leaders regarding the Ukraine peace proposal?",
        'ground_truth': "The criticism was triggered by a renewed peace proposal — reportedly shaped by a U.S. envoy — suggesting border changes in Ukraine. ",
        'context': """Ukraine Peace Moves Under Scrutiny: European leaders responded critically to a renewed peace proposal — reportedly shaped by a U.S. envoy — suggesting border changes in Ukraine. Vladimir Putin, whose forces are engaged in the conflict, acknowledged reviewing parts of the plan. Still, several officials demanded any deal must ensure a “just and lasting” peace rather than mere concessions."""
    },
    {
        'question': "What did some officials insist on even though parts of the proposal were being reviewed by Putin?",
        'ground_truth': "They insisted that any deal must ensure a “just and lasting” peace rather than mere concessions. ",
        'context': """Ukraine Peace Moves Under Scrutiny: European leaders responded critically to a renewed peace proposal — reportedly shaped by a U.S. envoy — suggesting border changes in Ukraine. Vladimir Putin, whose forces are engaged in the conflict, acknowledged reviewing parts of the plan. Still, several officials demanded any deal must ensure a “just and lasting” peace rather than mere concessions."""
    },
    {
        'question': "Where was the 2025 G20 Summit held and why was it significant?",
        'ground_truth': "The 2025 G20 Summit was held in Johannesburg — significant because it was the first held on African soil. ",
        'context': """G20 Summit Ends with Focus on Global South: The 2025 2025 G20 Johannesburg Summit, the first held on African soil, concluded with a 122-point declaration emphasizing global equity, debt relief for low-income nations, food security and multilateral reform. The decision signals a shift toward inclusive development and recognition of structural challenges faced by emerging economies."""
    },
    {
        'question': "What were some key themes of the declaration at the G20 Johannesburg Summit 2025?",
        'ground_truth': "The declaration emphasized global equity, debt relief for low-income nations, food security and multilateral reform. ",
        'context': """G20 Summit Ends with Focus on Global South: The 2025 2025 G20 Johannesburg Summit, the first held on African soil, concluded with a 122-point declaration emphasizing global equity, debt relief for low-income nations, food security and multilateral reform. The decision signals a shift toward inclusive development and recognition of structural challenges faced by emerging economies."""
    },
    {
        'question': "What triggered the massive fire in the Hong Kong high-rise residential complex?",
        'ground_truth': "Preliminary investigations point to flammable bamboo scaffolding and mesh — traditional building aids — as a possible factor in the blaze’s rapid spread. ",
        'context': """HK High-Rise Blaze: A massive fire engulfed a high-rise residential complex in Hong Kong, killing at least 44 people and leaving hundreds missing. Rescue operations are ongoing. Preliminary investigations point to flammable bamboo scaffolding and mesh — traditional building aids — as a possible factor in the blaze’s rapid spread."""
    },
    {
        'question': "Approximately how many people died in the Hong Kong high-rise blaze?",
        'ground_truth': "At least 44 people were killed. ",
        'context': """HK High-Rise Blaze: A massive fire engulfed a high-rise residential complex in Hong Kong, killing at least 44 people and leaving hundreds missing. Rescue operations are ongoing. Preliminary investigations point to flammable bamboo scaffolding and mesh — traditional building aids — as a possible factor in the blaze’s rapid spread."""
    },
    {
        'question': "What caused the suspension of processing immigration requests from Afghan nationals in Washington, D.C.?",
        'ground_truth': "It was stopped after two National Guard members were shot near the White House by a suspected Afghan national — the suspected gunman was taken into custody. ",
        'context': """US Domestic Security Alert: Authorities in Washington, D.C. stopped processing immigration requests from Afghan nationals after two National Guard members were shot near the White House. The suspected gunman, an Afghan national, was taken into custody. The incident triggered increased security and a temporary pause on asylum processing."""
    },
    {
        'question': "What was the immediate security action taken following the shooting near the White House?",
        'ground_truth': "They temporarily paused asylum / immigration processing for Afghan nationals and increased security. ",
        'context': """US Domestic Security Alert: Authorities in Washington, D.C. stopped processing immigration requests from Afghan nationals after two National Guard members were shot near the White House. The suspected gunman, an Afghan national, was taken into custody. The incident triggered increased security and a temporary pause on asylum processing."""
    },
    {
        'question': "Why did spot gold prices surge recently, according to the news?",
        'ground_truth': "Spot gold surged because of weak U.S. economic data and growing expectations that the Federal Reserve may soon lower interest rates, prompting investors to seek gold as a safe-haven asset. ",
        'context': """Gold Prices Surge as Investors Eye Fed Rate Cuts: Spot gold climbed about 1% to reach its highest level in two weeks, driven by weak U.S. economic data and growing expectations that the Federal Reserve (Fed) may soon lower interest rates, prompting investors to seek the metal as a safe-haven asset."""
    },
    {
        'question': "By how much did spot gold move to reach its recent high?",
        'ground_truth': "It climbed about 1%. ",
        'context': """Gold Prices Surge as Investors Eye Fed Rate Cuts: Spot gold climbed about 1% to reach its highest level in two weeks, driven by weak U.S. economic data and growing expectations that the Federal Reserve (Fed) may soon lower interest rates, prompting investors to seek the metal as a safe-haven asset."""
    },
    {
        'question': "By how much did Taiwan plan to increase its defence spending according to the announcement?",
        'ground_truth': "By US$ 40 billion. ",
        'context': """Taiwan’s Defence Budget Hike: The government in Taipei announced plans to increase defence spending by US$ 40 billion to counter escalated pressure from neighbouring China. The announcement comes amid rising tensions in the Taiwan Strait and growing concern over regional security dynamics."""
    },
    {
        'question': "What is the reason given for Taiwan’s defence budget hike?",
        'ground_truth': "The hike is to counter escalated pressure from neighbouring China amid rising tensions in the Taiwan Strait and growing concern over regional security dynamics. ",
        'context': """Taiwan’s Defence Budget Hike: The government in Taipei announced plans to increase defence spending by US$ 40 billion to counter escalated pressure from neighbouring China. The announcement comes amid rising tensions in the Taiwan Strait and growing concern over regional security dynamics."""
    },
    {
        'question': "How many sports are being contested at the 5th Khelo India University Games 2025 in Rajasthan?",
        'ground_truth': "Athletes are competing in 24 sports. ",
        'context': """Sports Spotlight: 5th Khelo India University Games 2025 underway in Rajasthan — with athletes from across the country competing in 24 sports. The games aim to provide a national platform for university-level talent and spotlight sports such as athletics, badminton, hockey, wrestling and kabaddi."""
    },
    {
        'question': "What are some of the sports featured at the 5th Khelo India University Games 2025?",
        'ground_truth': "Some of the sports include athletics, badminton, hockey, wrestling and kabaddi. ",
        'context': """Sports Spotlight: 5th Khelo India University Games 2025 underway in Rajasthan — with athletes from across the country competing in 24 sports. The games aim to provide a national platform for university-level talent and spotlight sports such as athletics, badminton, hockey, wrestling and kabaddi."""
    },
    {
        'question': "What were the winning numbers of the latest UK National Lottery draw and what was the jackpot prize?",
        'ground_truth': "The winning numbers were 10, 21, 49, 54, 55, 56 (with bonus ball 40); the jackpot prize was £2 million. ",
        'context': """Lottery Luck in UK: The latest draw of the National Lottery (UK) announced the winning numbers: 10, 21, 49, 54, 55, 56 (with bonus ball 40). The Thunderball numbers drawn were 15, 27, 30, 37, 39, with Thunderball 10. The jackpot prize stood at £2 million, reviving dreams of instant wealth for hopeful players."""
    },
    {
        'question': "Which set of Thunderball numbers were drawn in the latest UK National Lottery?",
        'ground_truth': "They were 15, 27, 30, 37, 39, with Thunderball 10. ",
        'context': """Lottery Luck in UK: The latest draw of the National Lottery (UK) announced the winning numbers: 10, 21, 49, 54, 55, 56 (with bonus ball 40). The Thunderball numbers drawn were 15, 27, 30, 37, 39, with Thunderball 10. The jackpot prize stood at £2 million, reviving dreams of instant wealth for hopeful players."""
    },
    {
        'question': "How many films and from how many countries are showcased at the 56th International Film Festival of India in Goa?",
        'ground_truth': "Over 240 films from 81 countries are featured. ",
        'context': """Cinema Celebration in Goa: The ongoing 56th International Film Festival of India features over 240 films from 81 countries and celebrates Japan as its “Country of Focus.” The opening ceremony was organized as a carnival parade — a first for the festival. Several world-premiere films and masterclasses by prominent directors mark this edition as especially ambitious."""
    },
    {
        'question': "Which country is celebrated as 'Country of Focus' at this year's International Film Festival of India in Goa?",
        'ground_truth': "Japan is the 'Country of Focus'. ",
        'context': """Cinema Celebration in Goa: The ongoing 56th International Film Festival of India features over 240 films from 81 countries and celebrates Japan as its “Country of Focus.” The opening ceremony was organized as a carnival parade — a first for the festival. Several world-premiere films and masterclasses by prominent directors mark this edition as especially ambitious."""
    },
    {
        'question': "How many stolen mobile phones were recovered across India via the Sanchar Saathi portal in October alone?",
        'ground_truth': "Over 50,000 stolen mobile phones were recovered in October alone via the portal. ",
        'context': """Tech and Telecom Crackdown Success: The national portal Sanchar Saathi, launched by the government’s telecom department, helped recover over 50,000 stolen mobile phones across India in October alone. Monthly recoveries surged 47% between June and October 2025, with states like Karnataka and Telangana each reporting over 100,000 recoveries since the portal’s inception."""
    },
    {
        'question': "By how much did monthly recoveries increase between June and October 2025 for the Sanchar Saathi portal?",
        'ground_truth': "Monthly recoveries surged 47% between June and October 2025. ",
        'context': """Tech and Telecom Crackdown Success: The national portal Sanchar Saathi, launched by the government’s telecom department, helped recover over 50,000 stolen mobile phones across India in October alone. Monthly recoveries surged 47% between June and October 2025, with states like Karnataka and Telangana each reporting over 100,000 recoveries since the portal’s inception."""
    }
]


In [11]:
qa_pairs[0]

{'question': 'Who did the soldiers arrest in the West African nation in the recent coup?',
 'ground_truth': 'They arrested President Embalo. Soldiers announced they had taken “total control” and arrested him. ',
 'context': 'Guinea-Bissau Coup: Soldiers in the West African nation announced they have taken “total control,” arresting President Embalo and suspending the electoral process. Military sources claim the detained president is being “well-treated,” while soldiers patrol capital streets. International reactions include calls for constitutional order and urgent humanitarian oversight.'}

## Generate answers

In [12]:
if not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

print(llm.invoke("Hi").content)

Hello! How can I assist you today?


In [23]:
# Generate Answers

from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """Answer the question only based on the context given.
    Answer only what is asked, in short, but full sentence.
    Question: {question}
    Context: {context}
    Answer:"""
)

def get_answer(question, context):
  prompt = prompt_template.format(
      question=question,
      context=context
  )
  resp = llm.invoke(prompt).content
  return resp

for p in range(len(qa_pairs)):
  print(qa_pairs[p]['question'])
  print(qa_pairs[p]['context'])
  answer = get_answer(qa_pairs[p]['question'], qa_pairs[p]['context'])
  print(answer)
  qa_pairs[p]['answer'] = answer
  # break

Who did the soldiers arrest in the West African nation in the recent coup?
Guinea-Bissau Coup: Soldiers in the West African nation announced they have taken “total control,” arresting President Embalo and suspending the electoral process. Military sources claim the detained president is being “well-treated,” while soldiers patrol capital streets. International reactions include calls for constitutional order and urgent humanitarian oversight.
The soldiers arrested President Embalo in the West African nation.
What did the military in Guinea-Bissau do after declaring total control?
Guinea-Bissau Coup: Soldiers in the West African nation announced they have taken “total control,” arresting President Embalo and suspending the electoral process. Military sources claim the detained president is being “well-treated,” while soldiers patrol capital streets. International reactions include calls for constitutional order and urgent humanitarian oversight.
After declaring total control, the milita

In [24]:
qa_pairs[0]

{'question': 'Who did the soldiers arrest in the West African nation in the recent coup?',
 'ground_truth': 'They arrested President Embalo. Soldiers announced they had taken “total control” and arrested him. ',
 'context': 'Guinea-Bissau Coup: Soldiers in the West African nation announced they have taken “total control,” arresting President Embalo and suspending the electoral process. Military sources claim the detained president is being “well-treated,” while soldiers patrol capital streets. International reactions include calls for constitutional order and urgent humanitarian oversight.',
 'answer': 'The soldiers arrested President Embalo in the West African nation.'}

## Save the answers

In [25]:
pickle.dump(news_docs,open('news_docs.pkl','wb'))
pickle.dump(qa_pairs,open('qa_pairs.pkl','wb'))

In [ ]:
# qa_pairs_loaded = pickle.load(open('qa_pairs.pkl','rb'))
# news_docs_loaded = pickle.load(open('news_docs.pkl','rb'))

In [26]:
data_samples = {
    'question': [p['question'] for p in qa_pairs],
    'answer': [p['answer'] for p in qa_pairs],
    'contexts': [[p['context']] for p in qa_pairs],
    'ground_truth': [p['ground_truth'] for p in qa_pairs]
}
data_samples_ds = Dataset.from_dict(data_samples)

In [27]:
len(data_samples_ds)

40

In [29]:
# check the data that is going in evaluate
data_samples_ds[0]

{'question': 'Who did the soldiers arrest in the West African nation in the recent coup?',
 'answer': 'The soldiers arrested President Embalo in the West African nation.',
 'contexts': ['Guinea-Bissau Coup: Soldiers in the West African nation announced they have taken “total control,” arresting President Embalo and suspending the electoral process. Military sources claim the detained president is being “well-treated,” while soldiers patrol capital streets. International reactions include calls for constitutional order and urgent humanitarian oversight.'],
 'ground_truth': 'They arrested President Embalo. Soldiers announced they had taken “total control” and arrested him. '}

## Evaluate using RAGAS on faithfulness and correctness

In [30]:
score = evaluate(data_samples_ds, metrics=[faithfulness, answer_correctness])

Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

In [31]:
score

{'faithfulness': 1.0000, 'answer_correctness': 0.8390}

In [32]:
score_df = score.to_pandas()

In [33]:
score_df.to_excel("evaluation.xlsx")

In [34]:
score_df.head()

,user_input,retrieved_contexts,response,reference,faithfulness,answer_correctness
0,Who did the soldiers arrest in the West Africa...,[Guinea-Bissau Coup: Soldiers in the West Afri...,The soldiers arrested President Embalo in the ...,They arrested President Embalo. Soldiers annou...,1.0,0.733682
1,What did the military in Guinea-Bissau do afte...,[Guinea-Bissau Coup: Soldiers in the West Afri...,"After declaring total control, the military in...",They suspended the electoral process.,1.0,0.597869
2,How many students were freed in the Nigerian s...,[Nigeria Freed 24 Kidnapped Students: Authorit...,Twenty-four students were freed in the Nigeria...,24 students were freed.,1.0,0.731630
3,After how many days were the kidnapped Nigeria...,[Nigeria Freed 24 Kidnapped Students: Authorit...,The kidnapped Nigerian students were rescued a...,They were rescued three days after being taken.,1.0,0.980336
4,What change did the IMF make regarding India’s...,[International Monetary Fund (IMF) has reclass...,The IMF reclassified India’s exchange-rate reg...,The IMF reclassified India’s exchange-rate reg...,1.0,0.744047


## Learning

- RAGAS makes the evaluation process quite easy
- Here we assumed the retrieval prcoess is perfect
- Correctness tells if the answer is related to Ground Truth
- Faithfullness tells if the answer is based on the context provided
- Since there is only one short document provided in the context, faithfulness is mostly 1
- In many cases generated answer is correct, but still correctness metric is not 1, so even if the correctness metric is >0.8, we can consider the system good.